<a href="https://colab.research.google.com/github/maddogmikeb/Jira/blob/master/UltimateSprintReport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# install dependencies

from IPython.core.display import clear_output

!pip install -q atlassian-python-api
!pip install -q matplotlib
!pip install -q numpy

clear_output()

In [47]:
from atlassian import Jira
from google.colab import userdata
from urllib.parse import urlparse
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import base64
import io
import re
import json
from datetime import datetime

class DataPoint:
    def __init__(self, name, count, points, color, hatch, edge_color):
        self.name = name
        self.count = count
        self.points = points
        self.color = color
        self.hatch = hatch
        self.edge_color = edge_color

    def get_values(self):
        return [self.count, self.points]

class UltimateSprintReport:

  def __init__(self, username, password, sprint_report_url):
    self.base_url, self.project, self.rapidViewId, self.sprintId = self._parse_url(sprint_report_url)

    self.jira = jira = Jira(
        url=self.base_url,
        username=username,
        password=password,
        cloud=True
      )

  def load():
    self._load_status_categories()
    self._load_sprint_report()
    self._load_velocity_statistics()
    self._load_board_config()
    self._load_sprint_statistics()
    self._load_committed_vs_planned_chart()
    self._calculate_sprint_details()
    self._calculate_sprint_predictability()
    self._calculate_epic_statistics()

  def _calculate_epic_statistics(self):
    epic_stats = []

    estimation_field = self.board_config['estimationStatisticConfig']['currentEstimationStatistic']['id'].replace('field_', '', 1)

    epics_being_worked_on = []
    for issue in self.sprint_report['contents']['completedIssues']:
      if issue['typeName'] == "Epic":
        epics_being_worked_on.append(issue['key'])
      elif 'epic' in issue:
        epics_being_worked_on.append(issue['epic'])

    for epic_key in list(set(epics_being_worked_on)):
      epic = self.jira.issue(key=epic_key)
      issues_in_epic = self.jira.jql(
        jql='issue in portfolioChildIssuesOf("' + epic_key + '")',
        fields=','.join(["status", estimation_field])
      )
      total_pts = 0
      total_cnt = 0
      done_pts = 0
      done_cnt = 0
      for issue in issues_in_epic['issues']:
        if issue['fields'][estimation_field]:
          total_pts += issue['fields'][estimation_field]
          if (issue['fields']["status"]["statusCategory"]["name"] == "Done"):
            done_pts += issue['fields'][estimation_field]
        total_cnt += 1
        if (issue['fields']["status"]["statusCategory"]["name"] == "Done"):
          done_cnt += 1

      epic_stats.append(dict(
        parent_key = epic['fields']['parent']['key'] if 'parent' in epic['fields'] and epic['fields']['parent'] else None,
        parent_summary = epic['fields']['parent']['fields']['summary'] if 'parent' in epic['fields'] and epic['fields']['parent'] else None,
        key = epic['key'],
        summary = epic['fields']['summary'],
        status_category = epic['fields']['status']['statusCategory']['name'] if epic['fields']['status']['statusCategory'] and 'name' in epic['fields']['status']['statusCategory'] else "To Do",
        done_pts = done_pts,
        total_pts = total_pts,
        completed_pts_perc = done_pts / total_pts * 100,
        done_cnt = done_cnt,
        total_cnt = total_cnt,
        completed_cnt_perc = done_cnt / total_cnt * 100,
      ))

    self.epic_statistics = epic_stats

  def _calculate_sprint_details(self):
    start = datetime.strptime(self.sprint_report["sprint"]["isoStartDate"], "%Y-%m-%dT%H:%M:%S%z").date()
    end = datetime.strptime(self.sprint_report["sprint"]["isoEndDate"], "%Y-%m-%dT%H:%M:%S%z").date()
    weekmask = ' '.join([k.capitalize()[:3] for k,v in dict(self.board_config['workingDaysConfig']['weekDays']).items() if v == True])
    holidays = [datetime.strptime(date, "%Y-%m-%d").date() for date in [x['iso8601Date'] for x in self.board_config['workingDaysConfig']['nonWorkingDays']]]
    days = np.busday_count(start, end, holidays=holidays, weekmask=weekmask)
    if (days > 1):
      days = days + 1 #include the start day

    self.sprint_details = dict(
      name = str(self.sprint_report["sprint"]["name"]) ,
      goal = str(self.sprint_report["sprint"]["goal"]),
      start_date_string = str(self.sprint_report["sprint"]["startDate"]),
      start_date = start,
      end_date_string = str(self.sprint_report["sprint"]["endDate"]),
      duration_days = str(days)
    )

  def _cumulate_data(self, data_array):
    data = np.array(data_array)
    data_shape = data.shape

    def get_cumulated_array(data, **kwargs):
      cum = np.cumsum(data.clip(**kwargs), axis=0)
      d = np.zeros_like(data)
      d[1:] = cum[:-1]
      return d

    cumulated_data = get_cumulated_array(data, min=0)
    cumulated_data_neg = get_cumulated_array(data, max=0)

    row_mask = data < 0
    cumulated_data[row_mask] = cumulated_data_neg[row_mask]
    data_stack = cumulated_data

    return data, data_shape, data_stack

  def _plot_data(self, values, col_labels, cols, edge_colors, hatch, total_committed):
    data, data_shape, data_stack = self._cumulate_data(values)

    _, ax = plt.subplots(figsize=(10, 10))

    for i in range(data_shape[0]):
        bars = ax.bar(np.arange(data_shape[1]), data[i], bottom=data_stack[i], color=cols[i], edgecolor=edge_colors[i], hatch=hatch[i], width=0.5)
        for bar in bars:
            height = bar.get_height()
            if height > 0:
                ax.text(bar.get_x() + bar.get_width() / 2, bar.get_y() + height / 2, f'{int(height)}', ha='center', va='center', color='black', fontweight='bold')

    for i in range(len(total_committed)):
        ax.vlines(x=i-0.4, ymin=0, ymax=total_committed[i], color='#8590a2', linestyle='solid', linewidth=5)
        if total_committed[i] > 0:
            ax.text(i-0.4, total_committed[i] + 0.4, f'{str(total_committed[i]).rjust(3)}', color='black', horizontalalignment='center', fontweight='bold')

    ax.set_xticks(np.arange(data_shape[1]))
    ax.set_xticklabels(col_labels)

    ax.axhline(0, color='black', linewidth=0.8)

    legend_elements = [
        Line2D([0], [0], color='#8590a2', lw=2, label='Committed'),
        Patch(facecolor=self.ToDo.color, edgecolor=self.ToDo.edge_color, label=self.ToDo.name),
        Patch(facecolor=self.InProgress.color, edgecolor=self.InProgress.edge_color, label=self.InProgress.name),
        Patch(facecolor=self.Done.color, edgecolor=self.Done.edge_color, label=self.Done.name),
        Patch(facecolor=self.CompletedOutside.color, edgecolor=self.CompletedOutside.edge_color, hatch="X", label=self.CompletedOutside.name),
        Patch(facecolor=self.Removed.color, edgecolor=self.Removed.edge_color, label=self.Removed.name)
    ]

    ax.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left')

    ax.set_ylim(top=ax.get_ylim()[1] * 1.1)

    plt.tight_layout()

    buf = io.BytesIO()
    plt.savefig(buf, format='png', pad_inches=0.5)
    buf.seek(0)
    image_base64 = base64.b64encode(buf.read()).decode('utf-8')
    buf.close()
    plt.close()

    return f'<img src="data:image/png;base64,{image_base64}" width="400px" height="400px" alt="Committed vs Planned"/>'

  def _calculate_predictability_score(self, estimated_points, completed_points):
    if estimated_points == 0:
      return  None, "-"

    predictability_score = abs(1 - (completed_points / estimated_points))
    if predictability_score <= 0.2:
      stars = "★★★★★"
    elif predictability_score <= 0.4:
      stars = "★★★★"
    elif predictability_score <= 0.6:
      stars = "★★★"
    elif predictability_score <= 0.8:
      stars = "★★"
    elif predictability_score <= 1.0:
      stars = "★"
    else:
      stars = "☆"
    return predictability_score, stars

  def _calculate_predictability(self, velocity_statistics, sprint_id, this_sprint_points_completed, this_sprint_points_committed):
    this_sprint_predictability = None
    predictability_data = []

    for sprint in sorted(velocity_statistics["sprints"], key=lambda i: -i["sequence"]):
      sprint_id_str = str(sprint["id"])
      estimated_points = velocity_statistics['velocityStatEntries'][sprint_id_str]["estimated"].get("value", 0)
      completed_points = velocity_statistics['velocityStatEntries'][sprint_id_str]["completed"].get("value", 0)

      predictability_score, stars = self._calculate_predictability_score(estimated_points, completed_points)

      sprint["predictability_score"] = predictability_score
      sprint["stars"] = stars

      if str(sprint_id) == sprint_id_str:
          this_sprint_predictability = {"predictability_score": predictability_score, "stars": stars}

      predictability_data.append({
          "name": sprint["name"],
          "estimated_points": estimated_points,
          "completed_points": completed_points,
          "predictability_score": predictability_score,
          "stars": stars
      })

    if not this_sprint_predictability:
      predictability_score, stars = self._calculate_predictability_score(this_sprint_points_committed, this_sprint_points_completed)
      this_sprint_predictability = {"predictability_score": predictability_score, "stars": stars + " (interim)"}

    return this_sprint_predictability, predictability_data

  def _calculate_sprint_predictability(self):
    self.this_sprint_predictability, self.predictability_data = self._calculate_predictability(
      self.velocity_statistics,
      self.sprintId,
      self.Done.points + self.CompletedOutside.points,
      self.TotalCommitted[1]
    )

  def _load_committed_vs_planned_chart(self):
    col_labels = ['Count', 'Points']
    cols = [self.Removed.color, self.Done.color, self.CompletedOutside.color, self.InProgress.color, self.ToDo.color]
    edge_colors = [self.Removed.edge_color, self.Done.edge_color, self.CompletedOutside.edge_color, self.InProgress.edge_color, self.ToDo.edge_color]
    hatch = [self.Removed.hatch, self.Done.hatch, self.CompletedOutside.hatch, self.InProgress.hatch, self.ToDo.hatch]
    values = [self.Removed.get_values(), self.Done.get_values(), self.CompletedOutside.get_values(), self.InProgress.get_values(), self.ToDo.get_values()]

    self.committed_vs_planned_chart = self._plot_data(values, col_labels, cols, edge_colors, hatch, self.TotalCommitted)

  def _get_status_category_id(self, name):
    return str(next(x['id'] for x in self.status_categories if x['name'] == name))

  def _calculate_estimates(self, status_category_id):
    count = 0
    estimate = 0

    if not self.sprint_report:
      raise Exception("Sprint Report not loaded")

    for issue in self.sprint_report["contents"]["issuesNotCompletedInCurrentSprint"]:
        if issue["status"]["statusCategory"]['id'] == status_category_id:
            count += 1
            estimate += issue['estimateStatistic']['statFieldValue'].get('value', 0)
    return count, estimate

  def _load_sprint_statistics(self):
    if not self.sprint_report:
      raise Exception("Sprint Report not loaded")

    TODO_KEY_ID = self._get_status_category_id('To Do')
    INPROGRESS_KEY_ID = self._get_status_category_id('In Progress')

    ToDoCount, ToDoEstimate = self._calculate_estimates(TODO_KEY_ID)
    ProgressCount, ProgressEstimate = self._calculate_estimates(INPROGRESS_KEY_ID)

    self.Removed = DataPoint(
        'Removed',
        -len(self.sprint_report["contents"]["puntedIssues"]),
        -self.sprint_report["contents"].get('puntedIssuesEstimateSum', {}).get('value', 0),
        '#d04437',
        None,
        '#ccc'
    )

    self.ToDo = DataPoint(
        'ToDo',
        ToDoCount,
        ToDoEstimate,
        '#091E420F',
        None,
        '#44546F'
    )

    self.InProgress = DataPoint(
        'InProgress',
        ProgressCount,
        ProgressEstimate,
        '#deebff',
        None,
        '#0055CC'
    )

    self.Done = DataPoint(
        'Completed',
        len(self.sprint_report["contents"]["completedIssues"]),
        self.sprint_report["contents"].get("completedIssuesEstimateSum", {}).get('value', 0),
        '#e3fcef',
        None,
        '#216E4E'
    )

    self.CompletedOutside = DataPoint(
        'Completed Outside',
        len(self.sprint_report["contents"]["issuesCompletedInAnotherSprint"]),
        self.sprint_report["contents"].get("issuesCompletedInAnotherSprintEstimateSum", {}).get('value', 0),
        '#e3fcef',
        'X',
        '#216E4E'
    )

    if self.sprint_velocity_statistics:
        self.TotalCommitted = [
            len(self.sprint_velocity_statistics["allConsideredIssueKeys"]),
            self.sprint_velocity_statistics["estimated"].get('value', 0)
        ]
    else:
        self.TotalCommitted = [
            len(self.sprint_report["contents"]["completedIssues"]) +
            len(self.sprint_report["contents"]["issuesNotCompletedInCurrentSprint"]) +
            len(self.sprint_report["contents"]["issuesCompletedInAnotherSprint"]) +
            len(self.sprint_report["contents"]["puntedIssues"]) -
            len(self.sprint_report["contents"]["issueKeysAddedDuringSprint"]),
            sum(
                float(self.sprint_report["contents"].get(key, {}).get('value', 0))
                for key in [
                    "completedIssuesInitialEstimateSum",
                    "issuesNotCompletedInitialEstimateSum",
                    "puntedIssuesInitialEstimateSum",
                    "issuesCompletedInAnotherSprintInitialEstimateSum"
                ]
            )
        ]


  def _load_board_config(self):
    self.board_config = json.loads(
        self.jira.request(
            absolute=True,
            method="GET",
            path="{base_url}{path}?rapidViewId={rapidViewId}".format(
                base_url = self.base_url,
                path="/rest/greenhopper/1.0/rapidviewconfig/editmodel.json",
                rapidViewId=self.rapidViewId
            )
          ).content
      )

  def _load_velocity_statistics(self):
    self.velocity_statistics = json.loads(
        self.jira.request(
            absolute=True,
            method="GET",
            path="{base_url}{path}?rapidViewId={rapidViewId}".format(
                base_url = self.base_url,
                path="/rest/greenhopper/1.0/rapid/charts/velocity.json",
                rapidViewId=self.rapidViewId
                )
            ).content
        )
    try:
      self.sprint_velocity_statistics = self.velocity_statistics['velocityStatEntries'][str(self.sprintId)]
    except:
      self.sprint_velocity_statistics = None

  def _load_sprint_report(self):
    self.sprint_report = json.loads(
        self.jira.request(
          absolute=True,
          method="GET",
          path="{base_url}{path}?rapidViewId={rapidViewId}&sprintId={sprintId}".format(
              base_url = self.base_url,
              path="/rest/greenhopper/latest/rapid/charts/sprintreport",
              rapidViewId=self.rapidViewId,
              sprintId=self.sprintId
            )
          ).content
        )

  def _load_status_categories(self):
    self.status_categories = self.jira.get("/rest/api/2/statuscategory")

  def _parse_url(self, url):
      pattern = r"(https?)://([^/]+)/jira/software/c/projects/([^/]+)/boards/(\d+)/reports/sprint-retrospective\?sprint=(\d+)"
      match = re.search(pattern, url)
      if match:
          protocol = match.group(1)
          base_url = match.group(2)
          project = match.group(3)
          rapidViewId = match.group(4)
          sprintId = match.group(5)
          full_base_url = f"{protocol}://{base_url}"
          return full_base_url, project, rapidViewId, sprintId
      else:
          return None, None, None, None

  def connected(self):
    try:
      me = self.jira.myself()
      return True
    except:
      pass
      return False

  def show_login_details(self):
    me = self.jira.myself()
    return """
      <table>
        <tr>
          <td>This report executed by:</td>
          <td>""" + me["displayName"] + """</td>
          <td><img src='""" + me["avatarUrls"]["32x32"] + """' /><td>
        </tr>
      </table>
    """

  def show_committed_vs_planned(self):
    return """
      <h2>Sprint Points & Issue Counts</h2>
      <table>
        <thead>
          <tr>
            <th>Category</th>
            <th>Count</th>
            <th>Points</th>
          </tr>
        </thead>
        <tbody>
          <tr>
            <td>Total Committed</td>
            <td align='right'>""" + str(self.TotalCommitted[0]) + """</td>
            <td align='right'>""" + f"{self.TotalCommitted[1]:.1f}" + """</td>
          </tr>
          <tr>
            <td colspan="3">
              <hr />
            </td>
          </tr>
          <tr>
            <td>Completed</td>
            <td align='right'>""" + str(self.Done.count) + """</td>
            <td align='right'>""" + f"{self.Done.points:.1f}" + """</td>
          </tr>
          <tr>
            <td>Completed Outside</td>
            <td align='right'>""" + str(self.CompletedOutside.count) + """</td>
            <td align='right'>""" + f"{self.CompletedOutside.points:.1f}" + """</td>
          </tr>
          <tr>
            <td>In Progress</td>
            <td align='right'>""" + str(self.InProgress.count) + """</td>
            <td align='right'>""" + f"{self.InProgress.points:.1f}" + """</td>
          </tr>
          <tr>
            <td>To Do</td>
            <td align='right'>""" + str(self.ToDo.count) + """</td>
            <td align='right'>""" + f"{self.ToDo.points:.1f}" + """</td>
          </tr>
          <tr>
            <td>Removed</td>
            <td align='right'>""" + str(self.Removed.count) + """</td>
            <td align='right'>""" + f"{self.Removed.points:.1f}" + """</td>
          </tr>
        </tbody>
      </table>
    """

  def show_sprint_details(self):
    return """
      <h2>Sprint Details</h2>
      <table>
        <tbody>
          <tr>
            <td>Sprint Name</td>
            <td>""" + self.sprint_details['name'] + """</td>
          </tr>
          <tr>
            <td>Sprint Goal</td>
            <td>""" + self.sprint_details['goal'] + """</td>
          </tr>
          <tr>
            <td>Start Date</td>
            <td>""" + self.sprint_details['start_date_string'] + """</td>
          </tr>
          <tr>
            <td>End Date</td>
            <td>""" + self.sprint_details['end_date_string'] + """</td>
          </tr>
          <tr>
            <td>Duration (days)</td>
            <td>""" + self.sprint_details['duration_days'] + """</td>
          </tr>
        </tbody>
      </table>
    """

  def show_predictability(self):
    predictability_data_table = """
      <h2>Predictability Statistics</h2>
      <table>
        <thead>
          <tr>
            <th>Sprint</th>
            <th>Estimated Points</th>
            <th>Completed Points</th>
            <th>Predictability Score</th>
            <th>Stars</th>
          </tr>
        </thead>
        <tbody>
    """

    for data in self.predictability_data:
        predictability_score = f"{data['predictability_score']:.2f}" if data['predictability_score'] is not None else "-"
        predictability_data_table += f"""
        <tr>
          <td>{data['name']}</td>
          <td align='right'>{data['estimated_points']}</td>
          <td align='right'>{data['completed_points']}</td>
          <td align='right'>{predictability_score}</td>
          <td align='right'>{data['stars']}</td>
        </tr>
        """

    predictability_data_table += """
      </tbody>
    </table>
    """
    return predictability_data_table

  def show_sprint_predictability(self):
    return f"""<h1> Rating: {self.this_sprint_predictability['stars']} </h1>""" if self.this_sprint_predictability else ""

  def show_epic_statistics(self):
    epic_statistics_table = """
      <h2>Epics Within Sprint Statistics</h2>
      <table>
        <thead>
          <tr>
            <th>Parent</th>
            <th>Epic</th>
            <th>Status</th>
            <th>Completed Points %</th>
            <th>Completed Count %</th>
          </tr>
        </thead>
        <tbody>
      """

    for epic in sorted(self.epic_statistics, key=lambda i: (i["parent_summary"] or "") + (i["summary"] or "")):
      parent = f"[{epic['parent_key']}] {epic['parent_summary']}" if epic['parent_key'] is not None else "-"
      epic_details = f"[{epic['key']}] {epic['summary']}"
      pts = f"{epic['completed_pts_perc']:.1f}" if epic['completed_pts_perc'] is not None else "-"
      cnt = f"{epic['completed_cnt_perc']:.1f}" if epic['completed_cnt_perc'] is not None else "-"
      epic_statistics_table += f"""
        <tr>
          <td>{parent}</td>
          <td>{epic_details}</td>
          <td>{epic['status_category']}</td>
          <td align='right'>{pts}</td>
          <td align='right'>{cnt}</td>
        </tr>
        """

    epic_statistics_table += """
      </tbody>
    </table>
    """
    return epic_statistics_table

  def show_report(self):
    return (
      self.show_login_details() +
      self.show_sprint_details() +
      self.show_predictability() +
      self.committed_vs_planned_chart +
      self.show_committed_vs_planned() +
      self.show_epic_statistics() +
      self.show_sprint_predictability()
    )
